# 유튜브 인기동영상 Data

In [2]:
import pandas as pd

# URL 형식으로 데이터 호출
dataUrl = 'https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv'
data = pd.read_csv(dataUrl,index_col = 0)
data.head()

title             channelTitle  \
0                                          [신병] 물자창고                      장삐쭈   
1   RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV  RAIN's Official Channel   
2       2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.              서울시 · Seoul   
3                                       고기남자의 칠면조 파티             고기남자 MeatMan   
4  골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...                    스브스밥집   

   categoryId  view_count  likes  dislikes  comment_count  \
0          23     1893473  38249       730           8595   
1          10     2600864      0         0          20129   
2          29      347049   3564       120            178   
3          26      528458  15372       280           3470   
4          24      494904   3918       111           3142   

                  channelId trending_date2  
0  UChbE5OZQ6dRHECsX0tEPEZQ     2021-01-01  
1  UCxXgIeE5hxWxHG6dz9Scg2w     2021-01-01  
2  UCZUPZW5idAxYp-Asj__lVAA     2021-01-01  
3  UCT3CumbFIJiW33uq0UI3zlg     2021-01-01  
4  UCdWgRSfttvDucq4ApcCg5Mw     2021-01-01

In [13]:
# 인기동영상 제작횟수 많은 채널 상위 10개 추출

data[data['channelId'].isin(data['channelId'].value_counts().head(10).index)].channelTitle.unique()

# data내에 channelId 값이 상위 10개의 id 값에 있는 값만 출력후, title 값으로 출력

array(['장삐쭈', '총몇명', '파뿌리', '짤툰', '런닝맨 - 스브스 공식 채널',
       '엠뚜루마뚜루 : MBC 공식 종합 채널', 'SPOTV', '채널 십오야', '이과장', 'BANGTANTV'],
      dtype=object)

In [17]:
# dislike수가 likes 수보다 높은 동영상 출력

data[data['dislikes']>data['likes']]['channelTitle'].unique()

array(['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang',
       '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담',
       'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈',
       '나혼자산다 STUDIO', 'Gen.G esports'], dtype=object)

In [36]:
# 채널명을 바꾼 케이스 ( channelId는 고유값)

data[['channelTitle','channelId']].duplicated().value_counts() # True 58556개 / False = 1843
chg = data[['channelTitle','channelId']].drop_duplicates().channelId.value_counts() 
print(len(chg[chg>1]))

71


In [45]:
# 특정요일에 인기있던 영상 카테고리

data.loc[:,'days']=pd.to_datetime(data['trending_date2']).dt.day_name() # 날짜형식으로 변환 후 요일별 이름 생성
data[data['days']=='Sunday']['categoryId'].value_counts().index[0] # 일요일에 해당하는 categoryId 값 카운트 후 호출

24

In [55]:
# 요일별 category 개수 출력

data.groupby(['categoryId','days']).size().unstack() # 데이터 프레임 형식으로 출력

days        Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
categoryId                                                                
1              243     263       255     274       246      257        234
2              120     105       119      99       128      119        129
10             833     837       776     830       890      894        917
15             187     215       198     217       207      208        207
17             633     668       592     636       682      708        706
19              90      92        87      91        92       89         85
20             283     298       296     289       282      285        291
22            1288    1373      1289    1337      1341     1375       1333
23             568     594       570     556       560      569        566
24            2976    3148      3066    3096      2954     3084       3090
25             444     453       422     437       470      452        468
26             369     378       364     363       375      394        385
27             183     205       183     199       194      194        212
28             171     160       173     167       166      161        165
29              12      10        10       9        13       11         12

In [63]:
# viewcount 대비 댓글수가 가장 높은 영상 확인  (view_count값이 0은 제외)

tmp = data[data['view_count']!=0] # view_count 0값 제외
len(tmp[tmp['view_count']==0]) # result = 0

tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성
tmp.sort_values('ratio',ascending=False).iloc[0].title # ratio 값 기준으로 내림차순 정렬 후 첫번째 값 title 선택

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2992716606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['comment_count']/tmp['view_count']).dropna() # viewcount대비 commennt_count 비율 값을 새로운 컬럼 생성


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

In [68]:
# viewcount 대비 댓글수가 가장 낮은 영상 확인  (view_count,ratio값이 0은 제외)
len(tmp[tmp['ratio']==0]) # result = 777
tmp = tmp[tmp['ratio']!=0] # 0 값 제거
len(tmp[tmp['ratio']==0]) # result = 0

tmp.sort_values('ratio',ascending=True).iloc[0].title # 가장 ratio값이 낮은 title 출력

'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

In [76]:
# like 대비 dislike 수가 가작 적은 영상(0 값은 제외)

len(data[(data['likes']==0)|(data['dislikes']==0)]) # result = 706
tmp = data[(data['likes']!=0)&(data['dislikes']!=0)] # 0값 제외
len(tmp[(tmp['likes']==0)|(tmp['dislikes']==0)]) # result = 0

tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()
tmp.sort_values('ratio',ascending=True).iloc[0].title

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_8840\2299923241.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[:,'ratio'] = (tmp['dislikes']/tmp['likes']).dropna()


'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

In [92]:
# 가장많은 트랜드 영상 제작한 채널 이름

# data[data['channelId']=='UChbE5OZQ6dRHECsX0tEPEZQ']
data[['channelId','channelTitle']].value_counts().index[0][1] 

'짤툰'

In [98]:
# 20회 이상 인기동영상 리스트에 포함된 동영상

(data[['title','channelId']].value_counts()>=20).sum()

40

# 유튜브 공범컨텐츠 동영상 데이터

In [215]:
import pandas as pd


channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

channelid   subcnt    viewcnt  videocnt  \
0  UCkQCwnkQfgSuPTTnw_Y7v7w  1310000  410238653       736   
1  UCkQCwnkQfgSuPTTnw_Y7v7w  1310000  412531322       736   
2  UCkQCwnkQfgSuPTTnw_Y7v7w  1310000  412531322       735   
3  UCkQCwnkQfgSuPTTnw_Y7v7w  1310000  412531322       737   
4  UCkQCwnkQfgSuPTTnw_Y7v7w  1320000  412531322       737   

                    ct channelname  
0  2021-09-30 03:01:03         꽈뚜룹  
1  2021-09-30 09:01:03         꽈뚜룹  
2  2021-09-30 15:01:03         꽈뚜룹  
3  2021-09-30 21:01:03         꽈뚜룹  
4  2021-10-01 03:01:04         꽈뚜룹

videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
0  c5JQp6xafqc  1667010    30474         706            0   6587   
1  c5JQp6xafqc  1669089    30495         707            0   6589   
2  c5JQp6xafqc  1674759    30522         711            0   6596   
3  c5JQp6xafqc  1677026    30555         712            0   6604   
4  c5JQp6xafqc  1681824    30585         713            0   6600   

                    ct videoname  
0  2021-10-10 15:20:03    공범 EP1  
1  2021-10-10 15:30:03    공범 EP1  
2  2021-10-10 15:40:02    공범 EP1  
3  2021-10-10 15:50:03    공범 EP1  
4  2021-10-10 16:00:03    공범 EP1

In [216]:
# 날짜타입으로 변겨 후 video 데이터에서 videoname의 각 value 값 확인

channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

video['videoname'].value_counts()

 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: videoname, dtype: int64

In [217]:
# 수집된 각 video의 가장 최신화 된 날짜의 viewcount 값 출력

tmp = video.drop_duplicates('videoname',keep='last')
tmp[['viewcnt','videoname','ct']].sort_values(['videoname','ct']).reset_index(drop=True)

viewcnt videoname                  ct
0  3180532    공범 EP1 2021-11-01 15:30:03
1  2199328    공범 EP2 2021-11-01 15:30:03
2  1671294    공범 EP3 2021-11-01 15:30:03
3  1818493    공범 EP4 2021-11-01 15:30:03
4  1503435    공범 EP5 2021-11-01 15:30:04
5  1750222    공범 EP6 2021-11-01 15:30:04
6  1630200    공범 EP7 2021-11-01 15:30:05
7  1289088    공범 EP8 2021-11-01 15:30:05

In [218]:
# channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 된 구독자 수 구하기

tmp = channel[channel['ct']>=pd.to_datetime('2021-10-03')].drop_duplicates('channelid',keep='first').sort_values(['ct','channelname']).reset_index(drop=True)
tmp[['channelname','subcnt']]

channelname   subcnt
0            논리왕 전기   922000
1             야전삽짱재   257000
2        츄정ChuJeong   322000
3        김농밀의 농밀한 삶     7520
4       릴펄 Lilpearl    10100
5   와글와글 WagleWagle    55000
6               조나단   215000
7           형사!탐정되다    14900
8     Balming Tiger    54300
9     곽토리 kwak tori   471000
10              꽈뚜룹  1330000

In [219]:
# 각 채널의 202-10-03 03:00:00~2021-11-01 15:00:00까지 구독자 수 증가량 구하기

end = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-11-01 15']
start = channel[channel['ct'].dt.strftime('%Y-%m-%d %H')=='2021-10-03 03']

end_data = end[['channelname','subcnt']].reset_index(drop=True)
start_data = start[['channelname','subcnt']].reset_index(drop=True)

end_data.columns = ['channelname','end_sub']
start_data.columns = ['channelname','start_sub']

m_data = pd.merge(start_data,end_data)
m_data.loc[:,'updown']  = m_data['end_sub']-m_data['start_sub']
m_data[['channelname','updown']]

channelname  updown
0               꽈뚜룹   70000
1             야전삽짱재   11000
2        츄정ChuJeong    1000
3            논리왕 전기  -11000
4   와글와글 WagleWagle       0
5               조나단   12000
6           형사!탐정되다   10300
7        김농밀의 농밀한 삶    1540
8       릴펄 Lilpearl   11000
9     곽토리 kwak tori   -2000
10    Balming Tiger    2500

In [220]:
# 수집간격 5분이하 20분 이상인 데이터 추출 ( 왜 실패??!!!)

tmp = video[video['videoname']==' 공범 EP1'].sort_values('ct').reset_index(drop=True)
tmp[(tmp['ct'].diff(1)>=datetime.timedelta(minutes=20))|(tmp['ct'].diff(1)<=datetime.timedelta(minutes=5))]

videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
721   c5JQp6xafqc  2228250    34559         849            0   7191   
722   c5JQp6xafqc  2228250    34559         849            0   7191   
1636  c5JQp6xafqc  2707933    37901         988            0   7410   

                      ct videoname  
721  2021-10-13 09:41:37    공범 EP1  
722  2021-10-13 09:41:37    공범 EP1  
1636 2021-10-19 18:20:03    공범 EP1

In [221]:
# 각 에피소드 별 시작 날짜 및 이름 묶어서 데이터프레임 출력

tmp = video[['videoname','ct']].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first').reset_index(drop=True) # 전처리 순서 집중하기
tmp['date']=tmp['ct'].dt.date # dt.date = > 'YYYY-MM-DD' 형식으로 출력(문자열)
tmp[['videoname','date']]

videoname        date
0    공범 EP1  2021-10-07
1    공범 EP2  2021-10-09
2    공범 EP3  2021-10-14
3    공범 EP4  2021-10-16
4    공범 EP5  2021-10-21
5    공범 EP6  2021-10-23
6    공범 EP7  2021-10-28
7    공범 EP8  2021-10-30

In [222]:
#“공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 
#viewcnt를 내림차순으로 정렬하여 출력

tmp = video[video['ct'].dt.hour==21].sort_values(['ct','videoname']).drop_duplicates('videoname',keep='first')
tmp[['videoname','viewcnt','ct']].sort_values('viewcnt',ascending=False).reset_index(drop=True)

videoname  viewcnt                  ct
0    공범 EP8   264029 2021-10-30 21:00:08
1    공범 EP7   252032 2021-10-28 21:00:04
2    공범 EP4   217674 2021-10-16 21:00:04
3    공범 EP6   213899 2021-10-23 21:00:06
4    공범 EP5   201179 2021-10-21 21:00:04
5    공범 EP2   148144 2021-10-09 21:00:03
6    공범 EP3   147183 2021-10-14 21:00:04
7    공범 EP1   117340 2021-10-07 21:00:03

In [223]:
# video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 
# videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬

tmp = video.sort_values(['ct','videoname'],ascending=False).drop_duplicates('videoname',keep='first')
tmp['ratio'] = (tmp['dislikecnt']/tmp['likecnt']).dropna()

tmp[['videoname','ratio']].sort_values('ratio',ascending=True).reset_index(drop=True)

videoname     ratio
0    공범 EP7  0.013636
1    공범 EP6  0.015881
2    공범 EP4  0.017971
3    공범 EP5  0.021128
4    공범 EP3  0.022360
5    공범 EP1  0.027118
6    공범 EP2  0.029844
7    공범 EP8  0.141357

In [224]:
#2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임 출력
def check(x):
    result = max(x) - min(x)
    return result

tmp = video[(video['ct']>=pd.to_datetime('2021-11-01 00:00:00'))&(video['ct']<=pd.to_datetime('2021-11-01 15:00:00'))].reset_index(drop=True)
tmp[['videoname','viewcnt']].groupby('videoname').agg(check)

viewcnt
videoname         
 공범 EP1      13298
 공범 EP2      10300
 공범 EP3       9927
 공범 EP4       9824
 공범 EP5      10824
 공범 EP6      14141
 공범 EP7      26949
 공범 EP8      89147

In [228]:
# video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 출력

tmp = video[(video.duplicated())==True]
tmp[['videoname','ct']]

videoname                  ct
722     공범 EP1 2021-10-13 09:41:37
3927    공범 EP2 2021-10-13 09:41:37

# 월드컵 출전선수 골기록

In [321]:
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
data.head()

Player  Goals                Years  Country
0  Miroslav Klose     16  2002-2006-2010-2014  Germany
1         Ronaldo     15       1998-2002-2006   Brazil
2     Gerd Muller     14            1970-1974  Germany
3   Just Fontaine     13                 1958   France
4            Pele     12  1958-1962-1966-1970   Brazil

In [322]:
# 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 득점수 데이터프레임 형태 출력

data.groupby('Country').sum().sort_values('Goals',ascending=False).iloc[:5]

Goals
Country         
Brazil       228
Germany      226
Argentina    135
Italy        123
France       115

In [323]:
# 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라

tmp = data.groupby('Country')['Player'].size().sort_values(ascending=False).head(5).to_frame()
tmp.columns=['cnt']
tmp

cnt
Country       
Brazil      81
Germany     78
Italy       60
Argentina   59
France      58

In [324]:
# Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다. 
#년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
cnt=0
for i in range(len(data['Years'].str.split('-'))):
    for j in range(len(data['Years'].str.split('-')[i])):
        if len(data['Years'].str.split('-')[i][j]) !=4:
            cnt+=1
print(cnt)

45


In [335]:
# Q23에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력

data['yearLst'] = data.Years.str.split('-')

def checkFour(x):
    for value in x:
        if len(str(value)) != 4:
            return False
        
    return True
    
df2 = data[data.check ==True].reset_index(drop=True)
print(df2.shape[0])

1250


In [350]:
df2

Player  Goals                Years     Country  check  \
0          Miroslav Klose     16  2002-2006-2010-2014     Germany   True   
1                 Ronaldo     15       1998-2002-2006      Brazil   True   
2             Gerd Muller     14            1970-1974     Germany   True   
3           Just Fontaine     13                 1958      France   True   
4                    Pele     12  1958-1962-1966-1970      Brazil   True   
...                   ...    ...                  ...         ...    ...   
1245        Josip Skoblar      1                 1962  Yugoslavia   True   
1246          Safet Susic      1            1982-1990  Yugoslavia   True   
1247  Aleksandar Tirnanic      1                 1930  Yugoslavia   True   
1248  Djordje Vujadinovic      1                 1930  Yugoslavia   True   
1249         Branko Zebec      1            1954-1958  Yugoslavia   True   

                       yearLst  LenCup  
0     [2002, 2006, 2010, 2014]       4  
1           [1998, 2002, 2006]       3  
2                 [1970, 1974]       2  
3                       [1958]       1  
4     [1958, 1962, 1966, 1970]       4  
...                        ...     ...  
1245                    [1962]       1  
1246              [1982, 1990]       2  
1247                    [1930]       1  
1248                    [1930]       1  
1249              [1954, 1958]       2  

[1250 rows x 7 columns]

In [342]:
# 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

df2['LenCup']=df2['yearLst'].str.len()
result = df2['LenCup'].value_counts()[4]
result

16

In [355]:
# Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라
len(df2[(df2['Country']=='Yugoslavia')&(df2['LenCup']==2)])

7

In [363]:
#2002년도에 출전한 전체 선수는 몇명인가?

len(df2[df2['Years'].str.contains('2002')])

156

In [373]:
# 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

len(df2[df2['Player'].str.lower().str.contains('carlos')])

13

In [379]:
#월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

df2[['Player','Goals']][df2['LenCup']==1].iloc[0].Player

'Just Fontaine'

In [385]:
#월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

df2[df2['LenCup']==1]['Country'].value_counts().index[0]

'Brazil'

# 따릉이 데이터

In [3]:
import pandas as pd
data =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
data.head()

대여일자  대여시간  대여소번호 대여구분코드 성별 연령대코드  이용건수    운동량   탄소량     이동거리  사용시간
0  2021-06-01     0   3541    정기권  F  ~10대     1   0.00  0.00     0.00     8
1  2021-06-01     0    765    정기권  F  ~10대     1  27.21  0.35  1526.81    19
2  2021-06-01     0   2637    정기권  F  ~10대     1  41.40  0.37  1608.56    18
3  2021-06-01     0   2919    정기권  F  ~10대     1   0.00  0.00     0.00    75
4  2021-06-01     0    549    정기권  F  ~10대     1  13.04  0.17   731.55     6

In [39]:
# 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력

rst = data['대여일자'].value_counts().to_frame().reset_index()
rst.columns = ['대여일자','cnt']
ans = rst[rst['cnt']==rst['cnt'].max()]['대여일자'][0]
display(rst)
print(ans)

대여일자    cnt
0   2021-06-04  56788
1   2021-06-02  56100
2   2021-06-08  55623
3   2021-06-14  54945
4   2021-06-09  54507
5   2021-06-05  52730
6   2021-06-07  52623
7   2021-06-11  52089
8   2021-06-12  48569
9   2021-06-01  48081
10  2021-06-06  47787
11  2021-06-13  43131
12  2021-06-10  36992
13  2021-06-03  18985

2021-06-04


In [50]:
# 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가
#이를 이용하여 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력


data['day_name']=pd.to_datetime(data['대여일자']).dt.day_name()
tmp=data.groupby('day_name').size().to_frame().reset_index()
tmp.columns=['day','cnt']
tmp.sort_values('cnt',ascending=False)

day     cnt
6  Wednesday  110607
0     Friday  108877
1     Monday  107568
5    Tuesday  103704
2   Saturday  101299
3     Sunday   90918
4   Thursday   55977

In [78]:
# 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라
# data.head()
data.groupby(['day_name','대여소번호'],as_index=False).size().sort_values(['day_name','size'],ascending=False)\ #코드가 길어서 \사용
.drop_duplicates('day_name',keep='first').reset_index(drop=True)

day_name  대여소번호  size
0  Wednesday    502   282
1    Tuesday    502   267
2   Thursday   2715   137
3     Sunday    502   372
4   Saturday    502   378
5     Monday    502   242
6     Friday    502   277

In [110]:
# 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라
#일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

a = data[data['대여구분코드'].str.contains('일일권')]['연령대코드'].value_counts().sort_index() # 일일권, 일일권(비회원) 포함한 연령별 합계
b = data['연령대코드'].value_counts().sort_index() # 연령별 총 합계
ratio = (a/b).sort_values(ascending=False)
print(ratio.index[0])

~10대


In [116]:
# 연령대별 평균 이동거리를 구하여라
data.groupby('연령대코드')['이동거리'].mean().to_frame(name='mean_dis')

mean_dis
연령대코드             
20대    3211.890552
30대    3341.443859
40대    3514.857416
50대    3593.668100
60대    3538.145737
70대~   3085.039641
~10대   2634.426279

In [120]:
data['대여일자']=pd.to_datetime(data['대여일자'])

In [151]:
# 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

# tmp = data[(data['연령대코드']=='20대')&(data['이동거리']>=data['이동거리'].mean())].sort_values(['대여일자','대여소번호'],ascending=False)\
# .reset_index(drop=True).iloc[:200] --> 오답인 이유 : 20대 데이터 "추출 후" 이동거리 평균에 따른 추출인데 난 20대이면서, 평균에 따른 데이터 추출

tw = data[data.연령대코드 =='20대'].reset_index(drop=True)
tw_mean = tw[tw.이동거리 >= tw.이동거리.mean()].reset_index(drop=True)
tw_mean['탄소량'] =tw_mean['탄소량'].astype('float')
target =tw_mean.sort_values(['대여일자','대여소번호'], ascending=False).reset_index(drop=True).iloc[:200].탄소량
result = round(target.sum()/len(target),3)
print(result)


1.613


In [173]:
#6월 7일 ~10대의 “이용건수”의 중앙값은?

data[(data['연령대코드']=='~10대')&(data['대여일자']==pd.to_datetime('2021-06-07'))]['이용건수'].median()

1.0

In [206]:
#평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후 
#각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라
tmp = data[(data['대여일자'].dt.weekday<5)&(data['대여시간'].isin([6,7,8]))].groupby(['대여시간','대여소번호'])\
.size().to_frame('이용횟수')
ans = tmp.sort_values(['대여시간','이용횟수'],ascending=False).groupby('대여시간').head(3)
ans

이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36

In [209]:
#이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

data[data['이동거리']>=data['이동거리'].mean()]['이동거리'].std()

5092.139707505305

In [214]:
# 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

m = data[data['성별'].isin(['M','m'])]['이동거리'].mean()
f = data[data['성별'].isin(['F','f'])]['이동거리'].mean()
ans = pd.DataFrame({'성별':['남','여'],'이동거리':[m,f]})
ans

성별         이동거리
0  남  3209.110871
1  여  3468.575025

# 전 세계 행복도 지표 데이터

In [216]:
import pandas as pd
data =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
data.head()

행복랭킹          나라명     점수  상대GDP  사회적지원  행복기대치  선택의 자유도    관대함  부패에 대한인식  \
0     1      Finland  7.769  1.340  1.587  0.986    0.596  0.153     0.393   
1     2      Denmark  7.600  1.383  1.573  0.996    0.592  0.252     0.410   
2     3       Norway  7.554  1.488  1.582  1.028    0.603  0.271     0.341   
3     4      Iceland  7.494  1.380  1.624  1.026    0.591  0.354     0.118   
4     5  Netherlands  7.488  1.396  1.522  0.999    0.557  0.322     0.298   

     년도  
0  2019  
1  2019  
2  2019  
3  2019  
4  2019

In [233]:
#데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라

data[data['행복랭킹']==10]['점수'].mean()

7.259

In [236]:
# 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다. 
# 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라
data[data['행복랭킹']<=50].groupby('년도')['점수'].mean().to_frame(name='평균 행복점수')

평균 행복점수
년도           
2018  6.64678
2019  6.67002

In [249]:
#2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라
data[data['년도']==2018][['점수','부패에 대한인식']].corr().iloc[0,1]

0.40529152271510027

In [265]:
# 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라
display(len(data[['행복랭킹','나라명']])) # 전체 데이터 수
display(len(data[['행복랭킹','나라명']].drop_duplicates())) # 중복을 제거 한 데이터 (여기서 중복 데이터는 행복랭킹이 변화하지 않은 데이터)
len(data[['행복랭킹','나라명']])- len(data[['행복랭킹','나라명']].drop_duplicates())

312

297

15

In [263]:
data[data[['행복랭킹','나라명']].duplicated()] # 중복된 데이터

행복랭킹                       나라명     점수  상대GDP  사회적지원  행복기대치  선택의 자유도  \
156     1                   Finland  7.632  1.305  1.592  0.874    0.681   
159     4                   Iceland  7.495  1.343  1.644  0.914    0.677   
163     8               New Zealand  7.324  1.268  1.601  0.876    0.669   
177    22                     Malta  6.627  1.270  1.525  0.884    0.645   
189    34                 Singapore  6.343  1.529  1.451  1.008    0.631   
208    53                    Latvia  5.933  1.148  1.454  0.671    0.363   
211    56                   Jamaica  5.890  0.819  1.493  0.693    0.575   
215    60                Kazakhstan  5.790  1.143  1.516  0.631    0.454   
220    65                      Peru  5.663  0.934  1.249  0.674    0.530   
231    76                 Hong Kong  5.430  1.405  1.290  1.030    0.524   
278   123                Mozambique  4.417  0.198  0.902  0.173    0.531   
294   139                      Togo  3.999  0.259  0.474  0.253    0.434   
298   143                Madagascar  3.774  0.262  0.908  0.402    0.221   
308   153                  Tanzania  3.303  0.455  0.991  0.381    0.481   
310   155  Central African Republic  3.083  0.024  0.000  0.010    0.305   

       관대함  부패에 대한인식    년도  
156  0.202     0.393  2018  
159  0.353     0.138  2018  
163  0.365     0.389  2018  
177  0.376     0.142  2018  
189  0.261     0.457  2018  
208  0.092     0.066  2018  
211  0.096     0.031  2018  
215  0.148     0.121  2018  
220  0.092     0.034  2018  
231  0.246     0.291  2018  
278  0.206     0.158  2018  
294  0.158     0.101  2018  
298  0.155     0.049  2018  
308  0.270     0.097  2018  
310  0.218     0.038  2018

In [290]:
#2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 
#컬럼명은 v1,v2,corr으로 표시하라

tmp = data[data['년도']==2019].corr().iloc[:-1,:-1].unstack().to_frame(name='corr').reset_index()
tmp.columns=['v1','v2','corr']
tmp[tmp['corr']!=1].sort_values('corr',ascending=False).drop_duplicates('corr').head(5).reset_index(drop=True)

v1     v2      corr
0  행복기대치  상대GDP  0.835462
1  상대GDP     점수  0.793883
2  행복기대치     점수  0.779883
3  사회적지원     점수  0.777058
4  사회적지원  상대GDP  0.754906

In [306]:
# 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라
data.groupby('년도').tail(5).groupby('년도').mean()[['점수']]

점수
년도          
2018  3.1800
2019  3.1408

In [310]:
# 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과 
#평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라

a = data[(data['년도']==2019)&(data['상대GDP']>=data['상대GDP'].mean())]['점수'].mean()
b = data[(data['년도']==2019)&(data['상대GDP']<=data['상대GDP'].mean())]['점수'].mean()
a-b

1.3904839572192538

In [337]:
# 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라

data[['년도','부패에 대한인식']].sort_values(['년도','부패에 대한인식'],ascending=False).groupby('년도').head(20).\
groupby('년도').mean()

부패에 대한인식
년도            
2018    0.3267
2019    0.3201

In [349]:
# 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라

len(set(data[(data['년도']==2018)&(data['행복랭킹']<=50)]['나라명'])-set(data[(data['년도']==2019)&(data['행복랭킹']<=50)]['나라명']))

4

# 지역구 에너지 소비량 데이터

In [389]:
import pandas as pd
data= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
data.head()

DateTime  Temperature  Humidity  Wind Speed  general diffuse flows  \
0  1/1/2017 0:00        6.559      73.8       0.083                  0.051   
1  1/1/2017 0:10        6.414      74.5       0.083                  0.070   
2  1/1/2017 0:20        6.313      74.5       0.080                  0.062   
3  1/1/2017 0:30        6.121      75.0       0.083                  0.091   
4  1/1/2017 0:40        5.921      75.7       0.081                  0.048   

   diffuse flows  Zone 1 Power Consumption  Zone 2  Power Consumption  \
0          0.119               34055.69620                16128.87538   
1          0.085               29814.68354                19375.07599   
2          0.100               29128.10127                19006.68693   
3          0.096               28228.86076                18361.09422   
4          0.085               27335.69620                17872.34043   

   Zone 3  Power Consumption  
0                20240.96386  
1                20131.08434  
2                19668.43373  
3                18899.27711  
4                18442.40964

In [390]:
#DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라

data['DateTime']=pd.to_datetime(data['DateTime'])
tmp = data
tmp.loc[:,'month']=tmp['DateTime'].dt.month
tmp.groupby('month').size().to_frame(name='cnt')

cnt
month      
1      4464
2      4032
3      4464
4      4320
5      4464
6      4320
7      4464
8      4464
9      4320
10     4464
11     4320
12     4320

In [398]:
# 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라

tmp = data[data['DateTime'].dt.month ==3]
tmp.groupby(tmp['DateTime'].dt.hour)['Temperature'].mean().sort_values().values[0]

11.506612903225806

In [399]:
# 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라

tmp = data[data['DateTime'].dt.month ==3]
tmp.groupby(tmp['DateTime'].dt.hour)['Temperature'].mean().sort_values(ascending=False).values[0]

18.393602150537635

In [402]:
data.columns

Index(['DateTime', 'Temperature', 'Humidity', 'Wind Speed',
       'general diffuse flows', 'diffuse flows', 'Zone 1 Power Consumption',
       'Zone 2  Power Consumption', 'Zone 3  Power Consumption', 'month'],
      dtype='object')

In [404]:
# Zone 1 Power Consumption 컬럼의 value값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 
# Humidity의 평균을 구하여라

data[data['Zone 1 Power Consumption']>data['Zone 2  Power Consumption']]['Humidity'].mean()

68.23624448055094

In [418]:
# 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라

data[['Zone 1 Power Consumption','Zone 2  Power Consumption','Zone 3  Power Consumption']].corr()

Zone 1 Power Consumption  \
Zone 1 Power Consumption                   1.000000   
Zone 2  Power Consumption                  0.834519   
Zone 3  Power Consumption                  0.750733   

                           Zone 2  Power Consumption  \
Zone 1 Power Consumption                    0.834519   
Zone 2  Power Consumption                   1.000000   
Zone 3  Power Consumption                   0.570932   

                           Zone 3  Power Consumption  
Zone 1 Power Consumption                    0.750733  
Zone 2  Power Consumption                   0.570932  
Zone 3  Power Consumption                   1.000000

In [427]:
#Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C,
#그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라

def chg(x):
    if x<10:
        return 'A'
    elif 10<=x<20:
        return 'B'
    elif 20<=x<30:
        return 'C'
    else:
        return 'D'
    
tmp = data['Temperature'].apply(lambda x: chg(x))
tmp.value_counts()

B    26993
C    21105
A     2874
D     1444
Name: Temperature, dtype: int64

In [430]:
# 6월 데이터중 12시의 Temperature의 표준편차를 구하여라
data[(data['DateTime'].dt.month==6)&(data['DateTime'].dt.hour==12)]['Temperature'].std()

2.049941782795103

In [431]:
# 6월 데이터중 12시의 Temperature의 분산을 구하여라

data[(data['DateTime'].dt.month==6)&(data['DateTime'].dt.hour==12)]['Temperature'].var()

4.202261312849164

In [443]:
# Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?

data[data['Temperature']>data['Temperature'].mean()].sort_values('Temperature')['Humidity'].values[3]

87.9

In [445]:
#Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때 4번째 행의 Humidity 값은?
data[data['Temperature']>=data['Temperature'].median()].sort_values('Temperature')['Humidity'].values[3]

80.3

# 포켓몬 정보 데이터

In [446]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
data.head()

#                   Name Type 1  Type 2  Total  HP  Attack  Defense  \
0  1              Bulbasaur  Grass  Poison    318  45      49       49   
1  2                Ivysaur  Grass  Poison    405  60      62       63   
2  3               Venusaur  Grass  Poison    525  80      82       83   
3  3  VenusaurMega Venusaur  Grass  Poison    625  80     100      123   
4  4             Charmander   Fire     NaN    309  39      52       43   

   Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0       65       65     45           1      False  
1       80       80     60           1      False  
2      100      100     80           1      False  
3      122      120     80           1      False  
4       60       50     65           1      False

In [451]:
#Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라

abs(data.groupby('Legendary')['HP'].mean()[0]-data.groupby('Legendary')['HP'].mean()[1])

25.55614861329147

In [454]:
#Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?

data['Type 2'].value_counts().index[0]

'Flying'

In [459]:
# 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?

data[data['Type 1']==data['Type 1'].value_counts().index[0]]['Attack'].mean()/data[data['Type 1']==data['Type 1'].value_counts().index[0]]['Defense'].mean()

1.0165238678090576

In [469]:
# 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?

# 1. groupby 해결법
data[data['Legendary']==True].groupby(['Generation','Legendary']).size().to_frame(name='cnt').sort_values('cnt',ascending=False).index[0][0]

#2 value_counts로 간단하게 해결하기

data[data['Legendary']==True]['Generation'].value_counts().index[0]

3

In [477]:
# HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중 가장 절댓값이 큰 두 변수와 그 값을 구하여라

tmp = data[['HP','Attack', 'Defense', 'Sp. Atk','Sp. Def','Speed']].corr().unstack().reset_index().rename(columns={0: "corr"})
tmp[tmp['corr']!=1].sort_values('corr',ascending=False).iloc[0]

level_0     Sp. Def
level_1     Defense
corr       0.510747
Name: 26, dtype: object

In [490]:
# 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라

data.sort_values(['Generation','Attack']).groupby('Generation').head(3)['Attack'].mean()

19.5

In [491]:
# 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라

data.sort_values(['Generation','Attack'],ascending=False).groupby('Generation').head(5)['Attack'].mean()

157.23333333333332

In [499]:
# 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?

data[['Type 1','Type 2']].value_counts().head(1)

Type 1  Type 2
Normal  Flying    24
dtype: int64

In [503]:
#한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?

tmp = data[['Type 1','Type 2']].value_counts()
len(tmp[tmp==1])

39

# 성인 건강검진 데이터

In [1]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
data.head()

성별코드  연령대코드(5세단위)  신장(5Cm단위)  체중(5Kg단위)  허리둘레  시력(좌)  시력(우)  청력(좌)  청력(우)  \
0    F           55        145         55  73.0    0.7    0.5    1.0    1.0   
1    M           40        180         55  74.0    1.5    1.5    1.0    1.0   
2    F           55        150         50  72.0    1.0    0.2    1.0    1.0   
3    M           40        170         85  89.0    1.0    0.8    1.0    1.0   
4    F           40        155         45  62.0    0.5    1.0    1.0    1.0   

   수축기혈압  ...   혈색소  요단백  혈청크레아티닌  (혈청지오티)AST  (혈청지오티)ALT  감마지티피  흡연상태  \
0  129.0  ...  12.9  1.0      0.9       172.0       209.0   15.0     0   
1  102.0  ...  15.6  1.0      0.8        30.0        19.0   23.0     1   
2  116.0  ...  14.1  1.0      1.0        31.0        19.0   10.0     0   
3  124.0  ...  16.7  1.0      1.0        26.0        38.0  108.0     1   
4  127.0  ...  12.9  1.0      0.7        20.0        11.0   13.0     0   

   구강검진수검여부  치아우식증유무  치석  
0         Y      0.0   Y  
1         Y      0.0   Y  
2         Y      0.0   N  
3         Y      1.0   Y  
4         Y      0.0   N  

[5 rows x 26 columns]

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   성별코드         44553 non-null  object 
 1   연령대코드(5세단위)  44553 non-null  int64  
 2   신장(5Cm단위)    44553 non-null  int64  
 3   체중(5Kg단위)    44553 non-null  int64  
 4   허리둘레         44553 non-null  float64
 5   시력(좌)        44553 non-null  float64
 6   시력(우)        44553 non-null  float64
 7   청력(좌)        44553 non-null  float64
 8   청력(우)        44553 non-null  float64
 9   수축기혈압        44553 non-null  float64
 10  이완기혈압        44553 non-null  float64
 11  식전혈당(공복혈당)   44553 non-null  float64
 12  총콜레스테롤       44553 non-null  float64
 13  트리글리세라이드     44553 non-null  float64
 14  HDL콜레스테롤     44553 non-null  float64
 15  LDL콜레스테롤     44553 non-null  float64
 16  혈색소          44553 non-null  float64
 17  요단백          44553 non-null  float64
 18  혈청크레아티닌      44553 non-null  float64
 19  (혈청지

In [29]:
# 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고, 
# 연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라

data['혈압차']= data['수축기혈압']-data['이완기혈압']
ans = data.groupby('연령대코드(5세단위)')['혈압차'].var().sort_values(ascending=False).index[4]
print(ans)

60


In [32]:
# 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다. 
# 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

data['WHtR'] = data['허리둘레']/data['신장(5Cm단위)']
a = len(data[(data['WHtR']>=0.58)&(data['성별코드']=='F')])
b = len(data[(data['WHtR']>=0.58)&(data['성별코드']=='M')])
ans = b/a
print(ans)

1.1693877551020408


# 자동차 보험가입 예측데이터

In [33]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
data.head()

id  Gender  Age  Driving_License  Region_Code  Previously_Insured  \
0  188957  Female   25                1         17.0                   1   
1  275631    Male   20                1         22.0                   0   
2  329036  Female   40                1         41.0                   0   
3  227288  Female   28                1         30.0                   1   
4  300441  Female   50                1         28.0                   0   

  Vehicle_Age Vehicle_Damage  Annual_Premium  Policy_Sales_Channel  Vintage  \
0    < 1 Year             No         38785.0                 152.0       59   
1    < 1 Year             No          2630.0                 160.0      113   
2    1-2 Year            Yes         33165.0                 124.0      188   
3    < 1 Year             No         29038.0                 152.0      233   
4    1-2 Year            Yes         31325.0                 124.0      258   

   Response  
0         0  
1         0  
2         0  
3         0  
4         0

In [38]:
# Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서
# Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라
tmp = data[data['Vehicle_Age']=='> 2 Years']
ans = tmp[tmp['Annual_Premium']>=data['Annual_Premium'].median()]['Vintage'].mean()
print(ans)

154.43647182359118


In [42]:
# vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라

data.groupby(['Vehicle_Age','Gender'])['Annual_Premium'].mean().unstack()

Gender             Female          Male
Vehicle_Age                            
1-2 Year     30762.245001  30413.088469
< 1 Year     29972.286702  30310.982212
> 2 Years    36108.366374  35303.870627

# 핸드폰 가격 예측데이터

In [43]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")
data.head()

battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0            842     0          2.2         0   1       0           7    0.6   
1           1021     1          0.5         1   0       1          53    0.7   
2            563     1          0.5         1   2       1          41    0.9   
3            615     1          2.5         0   0       0          10    0.8   
4           1821     1          1.2         0  13       1          44    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        188        2  ...         20       756  2549     9     7         19   
1        136        3  ...        905      1988  2631    17     3          7   
2        145        5  ...       1263      1716  2603    11     2          9   
3        131        6  ...       1216      1786  2769    16     8         11   
4        141        2  ...       1208      1212  1411     8     2         15   

   three_g  touch_screen  wifi  price_range  
0        0             0     1            1  
1        1             1     0            2  
2        1             1     0            2  
3        1             0     0            2  
4        1             1     0            1  

[5 rows x 21 columns]

In [58]:
# price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라
data.groupby(['price_range','n_cores']).size().to_frame(name='level').sort_values('level').groupby(level=0).tail(1)

level
price_range n_cores       
0           2           69
3           5           70
2           4           73
1           1           76

In [83]:
# price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라

tmp = data[data['price_range']==3].corr().iloc[:-1,:-1].unstack().sort_values(ascending=False)
ans = tmp.loc[tmp!=1].reset_index().iloc[1]
print(ans)

level_0          pc
level_1          fc
0          0.635166
Name: 1, dtype: object


# 비행탐승 경험 만족도 데이터

In [85]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")
data.head()

id  Gender   Customer Type  Age   Type of Travel     Class  \
0   55882    Male  Loyal Customer   42  Business travel  Business   
1   31906  Female  Loyal Customer   31  Business travel  Business   
2   85481    Male  Loyal Customer   51  Personal Travel       Eco   
3  102134    Male  Loyal Customer   44  Business travel  Business   
4   31580    Male  Loyal Customer   33  Business travel  Business   

   Flight Distance  Inflight wifi service  Departure/Arrival time convenient  \
0              473                      1                                  1   
1             3271                      4                                  4   
2              214                      5                                  0   
3             3097                      1                                  1   
4             3286                      1                                  1   

   Ease of Online booking  ...  Inflight entertainment  On-board service  \
0                       3  ...                       5                 5   
1                       4  ...                       3                 3   
2                       5  ...                       3                 4   
3                       3  ...                       4                 4   
4                       1  ...                       4                 2   

   Leg room service  Baggage handling  Checkin service  Inflight service  \
0                 5                 5                3                 5   
1                 5                 5                3                 4   
2                 4                 4                4                 4   
3                 4                 4                3                 4   
4                 5                 2                5                 1   

   Cleanliness  Departure Delay in Minutes  Arrival Delay in Minutes  \
0            3                           3                       0.0   
1            3                           1                       5.0   
2            3                           2                       0.0   
3            5                           0                       0.0   
4            4                           0                       0.0   

   satisfaction  
0     satisfied  
1     satisfied  
2     satisfied  
3     satisfied  
4     satisfied  

[5 rows x 24 columns]

In [96]:
data['satisfaction'].value_counts()

neutral or dissatisfied    47103
satisfied                  36020
Name: satisfaction, dtype: int64

In [211]:
#Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다 ‘satisfied’의 수가 더 높은 Class는 어디 인가?

tmp = data[data['Arrival Delay in Minutes'].isna()].groupby(['Class','satisfaction']).size().unstack()
tmp[tmp['neutral or dissatisfied']<tmp['satisfied']]

satisfaction  neutral or dissatisfied  satisfied
Class                                           
Business                           36         76

# 수질 음용성 여부 데이터

In [212]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")
data.head()

ph    Hardness        Solids  Chloramines     Sulfate  Conductivity  \
0  7.918150  214.186611  23823.492888     7.290878  341.173322    411.424483   
1  5.422446  205.266080  18542.957451     5.491963  306.702227    382.080129   
2  7.341547  187.672402  21273.457066     7.784003         NaN    332.084293   
3  9.056245  197.666301  17403.532167     7.688917  337.460176    414.766631   
4  5.039374  142.860598  40829.353167     7.271543         NaN    386.803057   

   Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       19.585002        25.057375   4.028958           0  
1       10.504023        67.493450   2.911751           1  
2       16.842334        55.019151   4.025644           0  
3       15.349869        63.696746   3.319354           0  
4       16.823773        52.297113   4.957420           0

In [235]:
# ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?
tmp=data[~data['ph'].isna()]
tmp[tmp['ph']<=tmp['ph'].quantile(.25)]['ph'].mean()

5.057093462441732

# 의료 비용 예측 데이터

In [237]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")
data.head()

age     sex     bmi  children smoker     region      charges
0   64  female  39.330         0     no  northeast  14901.51670
1   47  female  27.830         0    yes  southeast  23065.42070
2   52  female  33.300         2     no  southwest  10806.83900
3   33  female  22.135         1     no  northeast   5354.07465
4   30    male  31.400         1     no  southwest   3659.34600

In [240]:
# 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

smok = data[data['smoker']=='yes']['charges'].quantile(0.9)
n_smok = data[data['smoker']=='no']['charges'].quantile(0.9)
m_y = data[(data['smoker']=='yes')&(data['charges']>=smok)]['charges'].mean()
m_n = data[(data['smoker']=='no')&(data['charges']>=n_smok)]['charges'].mean()
ans = m_y-m_n
print(ans)

29297.954548156158


# 킹카운티 주거지 가격예측문제 데이터

In [241]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
data.head()

id             date     price  bedrooms  bathrooms  sqft_living  \
0  8961990160  20150413T000000  567500.0         3        2.5         2080   
1  9455200205  20140604T000000  525000.0         3        2.0         1540   
2  7853220670  20140918T000000  540000.0         3        2.5         2860   
3  3298201170  20141110T000000  350000.0         3        1.0          940   
4  7972604355  20140521T000000  218000.0         3        1.0         1020   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      4556     2.0           0     0  ...      8        2080              0   
1      7800     1.0           0     0  ...      8        1540              0   
2      8935     2.0           0     0  ...      8        2860              0   
3      7811     1.0           0     0  ...      6         940              0   
4      7874     1.0           0     0  ...      7        1020              0   

   yr_built  yr_renovated  zipcode      lat     long  sqft_living15  \
0      1999             0    98074  47.6036 -122.014           1530   
1      2004             0    98125  47.7041 -122.288           1510   
2      2004             0    98065  47.5336 -121.855           2650   
3      1959             0    98008  47.6195 -122.118           1180   
4      1956             0    98106  47.5175 -122.346           1290   

   sqft_lot15  
0        5606  
1        7800  
2        6167  
3        7490  
4        7320  

[5 rows x 21 columns]

In [247]:
# bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라

data[data['bedrooms']==data['bedrooms'].value_counts().index[0]]['price'].quantile(.9)-\
data[data['bedrooms']==data['bedrooms'].value_counts().index[0]]['price'].quantile(.1)

505500.0

# 대학원 입학가능성 데이터

In [248]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
data.head()

Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR  CGPA  \
0         246        328          110                  4  4.0  2.5  9.02   
1         100        323          113                  3  4.0  4.0  8.88   
2          79        296           95                  2  3.0  2.0  7.54   
3          53        334          116                  4  4.0  3.0  8.00   
4         444        321          114                  5  4.5  4.5  9.16   

   Research  Chance of Admit  
0         1             0.81  
1         1             0.79  
2         1             0.44  
3         1             0.78  
4         1             0.87

In [251]:
# Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때, 
# 랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

from sklearn.ensemble import RandomForestRegressor
df_t = data.drop([data.columns[0]],axis = 1)
x = df_t.drop([data.columns[-1]],axis=1)
y = df_t[df_t.columns[-1]]

model = RandomForestRegressor()

model.fit(x,y)

result = pd.DataFrame({'importance':model.feature_importances_},x.columns).sort_values('importance',ascending=False)
result

importance
CGPA                 0.812039
GRE Score            0.085441
TOEFL Score          0.032197
LOR                  0.024472
SOP                  0.024133
University Rating    0.013898
Research             0.007820

# 레드 와인 퀄리티 예측 데이터

In [252]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")
data.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            5.0              0.38         0.01             1.6      0.048   
1            5.0              0.42         0.24             2.0      0.060   
2            7.1              0.36         0.30             1.6      0.080   
3            7.6              0.29         0.49             2.7      0.092   
4            7.7              0.51         0.28             2.1      0.087   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 26.0                  60.0  0.99084  3.70       0.75   
1                 19.0                  50.0  0.99170  3.72       0.74   
2                 35.0                  70.0  0.99693  3.44       0.50   
3                 25.0                  60.0  0.99710  3.31       0.61   
4                 23.0                  54.0  0.99800  3.42       0.74   

   alcohol  quality  
0     14.0        6  
1     14.0        8  
2      9.4        5  
3     10.1        6  
4      9.2        5

In [269]:
# quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때 그값이 가장 큰 컬럼명을 구하여라
a = data[data['quality']==3].iloc[:,:-1].std()
b = data[data['quality']==8].iloc[:,:-1].std()
abs(a-b).sort_values(ascending=False).index[0]

'total sulfur dioxide'

# 약물 분류 데이터

In [270]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")
data.head()

Age Sex      BP Cholesterol  Na_to_K  Drug
0   20   F  NORMAL      NORMAL    9.281     4
1   24   M    HIGH      NORMAL    9.475     1
2   34   M  NORMAL        HIGH   22.456     0
3   17   M  NORMAL      NORMAL   10.832     4
4   72   M     LOW        HIGH   16.310     0

In [277]:
# 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

tmp = data[data['Sex']=='M']
tmp['age2'] = tmp['Age']//10 *10
tmp.groupby('age2')['Na_to_K'].mean().to_frame(name='Na_to_K')

C:\Users\SuperUser\AppData\Local\Temp\ipykernel_24692\395598717.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['age2'] = tmp['Age']//10 *10


Na_to_K
age2           
10    13.627000
20    18.260769
30    15.244143
40    13.897273
50    14.811273
60    14.284308
70    11.363857

# 사기회사 분류 데이터

In [278]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
data.head()

Sector_score LOCATION_ID  PARA_A  Score_A  Risk_A  PARA_B  Score_B  Risk_B  \
0          2.72          14    4.53      0.6   2.718   87.93      0.6  52.758   
1          3.41          16   12.68      0.6   7.608   41.00      0.6  24.600   
2          2.72          11    3.11      0.6   1.866  113.97      0.6  68.382   
3          3.41           8    1.12      0.4   0.448    0.00      0.2   0.000   
4         55.57           9    1.06      0.4   0.424    0.00      0.2   0.000   

    TOTAL  numbers  ...  RiSk_E  History  Prob  Risk_F  Score  Inherent_Risk  \
0   92.46      5.0  ...     0.4        0   0.2     0.0    4.0        108.362   
1   53.68      5.5  ...     0.4        0   0.2     0.0    4.4         41.936   
2  117.08      5.5  ...     0.4        0   0.2     0.0    4.4         88.832   
3    1.12      5.0  ...     0.4        0   0.2     0.0    2.2          1.848   
4    1.06      5.0  ...     0.4        0   0.2     0.0    2.2          1.824   

   CONTROL_RISK  Detection_Risk  Audit_Risk  Risk  
0           0.4             0.5     21.6724     1  
1           0.4             0.5      8.3872     1  
2           0.4             0.5     17.7664     1  
3           0.4             0.5      0.3696     0  
4           0.4             0.5      0.3648     0  

[5 rows x 27 columns]

In [279]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sector_score    620 non-null    float64
 1   LOCATION_ID     620 non-null    object 
 2   PARA_A          620 non-null    float64
 3   Score_A         620 non-null    float64
 4   Risk_A          620 non-null    float64
 5   PARA_B          620 non-null    float64
 6   Score_B         620 non-null    float64
 7   Risk_B          620 non-null    float64
 8   TOTAL           620 non-null    float64
 9   numbers         620 non-null    float64
 10  Score_B.1       620 non-null    float64
 11  Risk_C          620 non-null    float64
 12  Money_Value     619 non-null    float64
 13  Score_MV        620 non-null    float64
 14  Risk_D          620 non-null    float64
 15  District_Loss   620 non-null    int64  
 16  PROB            620 non-null    float64
 17  RiSk_E          620 non-null    flo

In [281]:
# 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라

data.groupby('Risk')[['Score_A','Score_B']].mean()

Score_A   Score_B
Risk                    
0     0.262234  0.225532
1     0.490164  0.444262

# 센서데이터 동작유형 분류 데이터

In [283]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
data.head()

motion_0  motion_1  motion_2  motion_3  motion_4  motion_5  motion_6  \
0      -6.0       8.0       5.0       6.0     -12.0     -27.0       5.0   
1     -32.0      -3.0      -4.0      -5.0      -1.0      16.0       0.0   
2      35.0      -2.0      -1.0       1.0     -25.0     -16.0      -3.0   
3     -15.0      -6.0      -3.0      -5.0      27.0      37.0      -2.0   
4      60.0       3.0       0.0      -7.0      -6.0     -10.0      -6.0   

   motion_7  motion_8  motion_9  ...  motion_55  motion_56  motion_57  \
0      24.0      -8.0      -5.0  ...       -1.0       -3.0       -6.0   
1      -5.0      42.0       2.0  ...      -17.0      -12.0       -1.0   
2     -20.0      10.0       1.0  ...        6.0      -65.0        6.0   
3      -2.0      19.0       5.0  ...        2.0       11.0        0.0   
4       7.0     -11.0      -4.0  ...      -12.0       17.0       -3.0   

   motion_58  motion_59  motion_60  motion_61  motion_62  motion_63  pose  
0       -9.0       -7.0       36.0       68.0        3.0       -3.0     1  
1       -1.0        0.0      -26.0      -11.0       -4.0       -9.0     1  
2      -10.0      -31.0      -19.0       -3.0      -47.0      -17.0     0  
3       -2.0       -4.0       -7.0       15.0        3.0       14.0     1  
4        1.0        2.0       -2.0       20.0       18.0       10.0     0  

[5 rows x 65 columns]

In [288]:
# pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?

tmp =data.groupby('pose').median().T
abs(tmp[0]-tmp[1]).sort_values()
tmp.tail(5)

pose         0    1
motion_59  0.0 -1.0
motion_60 -1.0  1.0
motion_61 -1.0  1.0
motion_62 -5.0 -1.0
motion_63 -1.0 -1.0

# 현대 차량 가격 분류 문제 데이터

In [289]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")
data.head()

model  year  price transmission  mileage fuelType  tax(£)   mpg  \
0   Tucson  2016  12795       Manual    36263   Diesel      30  61.7   
1      I10  2012   3995       Manual    22089   Petrol      20  61.4   
2      I30  2019  24000       Manual      555   Petrol     145  34.0   
3   Tucson  2017  12995       Manual    32147   Diesel      30  61.7   
4   Tucson  2018  14991    Semi-Auto    32217   Diesel     150  57.6   

   engineSize  
0         1.7  
1         1.2  
2         2.0  
3         1.7  
4         1.7

In [298]:
# 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?

data[data['model'].isin(data['model'].value_counts().index[:3])].groupby('model')[['price']].mean()

price
model                
 I10      7646.137891
 I30     11651.821759
 Tucson  15805.441373

# 당뇨여부판단 데이터

In [299]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
data.head()

Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            3      102             74              0        0  29.5   
1            2      144             58             33      135  31.6   
2            5      136             82              0        0   0.0   
3           13      145             82             19      110  22.2   
4            1      117             60             23      106  33.8   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.121   32        0  
1                     0.422   25        1  
2                     0.640   69        0  
3                     0.245   57        0  
4                     0.466   27        0

In [312]:
# Outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이를 구하여라

tmp = data.groupby('Outcome').mean().T
tmp['diff']=tmp[0]-tmp[1]
tmp[['diff']]

# 방법 2

tmp = data.groupby('Outcome').mean().diff().iloc[1,:]
print(tmp)

Outcome                        diff
Pregnancies               -1.574159
Glucose                  -29.943995
BloodPressure             -3.202079
SkinThickness             -2.952033
Insulin                  -33.398645
BMI                       -4.541437
DiabetesPedigreeFunction  -0.143646
Age                       -5.866939

# 넷플릭스 주식 데이터

In [313]:
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
data.head()

Date      Open      High       Low     Close  Adj Close     Volume
0  2002-05-23  1.156429  1.242857  1.145714  1.196429   1.196429  104790000
1  2002-05-24  1.214286  1.225000  1.197143  1.210000   1.210000   11104800
2  2002-05-28  1.213571  1.232143  1.157143  1.157143   1.157143    6609400
3  2002-05-29  1.164286  1.164286  1.085714  1.103571   1.103571    6757800
4  2002-05-30  1.107857  1.107857  1.071429  1.071429   1.071429   10154200

In [320]:
#매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라

data['Date'] = pd.to_datetime(data['Date'])

data[data['Date'].dt.month==5].groupby(data['Date'].dt.year)[['Open']].mean()

Open
Date            
2002    1.155833
2003    1.641497
2004    4.261143
2005    1.951905
2006    4.163571
2007    3.159351
2008    4.435034
2009    5.790571
2010   14.417071
2011   34.650272
2012   10.523247
2013   31.936429
2014   51.121292
2015   85.057429
2016   92.705715
2017  158.255455
2018  329.779541
2019  359.664548
2020  433.880499
2021  496.923996

# nba 선수 능력치 데이터

In [321]:
pd.set_option('display.max_columns',50)
import pandas as pd
data = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')
data.head()

Rk             Player Pos  Age   Tm   G  GS    MP   FG   FGA    FG%   3P  \
0   1   Precious Achiuwa   C   22  TOR  73  28  23.6  3.6   8.3  0.439  0.8   
1   2       Steven Adams   C   28  MEM  76  75  26.3  2.8   5.1  0.547  0.0   
2   3        Bam Adebayo   C   24  MIA  56  56  32.6  7.3  13.0  0.557  0.0   
3   4       Santi Aldama  PF   21  MEM  32   0  11.3  1.7   4.1  0.402  0.2   
4   5  LaMarcus Aldridge   C   36  BRK  47  12  22.3  5.4   9.7  0.550  0.3   

   3PA    3P%   2P   2PA    2P%   eFG%   FT  FTA    FT%  ORB  DRB   TRB  AST  \
0  2.1  0.359  2.9   6.1  0.468  0.486  1.1  1.8  0.595  2.0  4.5   6.5  1.1   
1  0.0  0.000  2.8   5.0  0.548  0.547  1.4  2.6  0.543  4.6  5.4  10.0  3.4   
2  0.1  0.000  7.3  12.9  0.562  0.557  4.6  6.1  0.753  2.4  7.6  10.1  3.4   
3  1.5  0.125  1.5   2.6  0.560  0.424  0.6  1.0  0.625  1.0  1.7   2.7  0.7   
4  1.0  0.304  5.1   8.8  0.578  0.566  1.9  2.2  0.873  1.6  3.9   5.5  0.9   

   STL  BLK  TOV   PF   PTS  
0  0.5  0.6  1.2  2.1   9.1  
1  0.9  0.8  1.5  2.0   6.9  
2  1.4  0.8  2.6  3.1  19.1  
3  0.2  0.3  0.5  1.1   4.1  
4  0.3  1.0  0.9  1.7  12.9

In [322]:
# Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라

ans = round(data[data['Tm']=='TOR']['Age'].mean(),4)
print(ans)

24.8696


In [329]:
# Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라
tmp = data

tmp['age2'] = tmp['Age']//10*10
tmp[tmp['age2']==tmp['age2'].min()].groupby('Pos').size().sort_values(ascending=False).index[0]

'SG'

In [345]:
# 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다. 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라

ans = data['Player'].str.split().str[0].str.lower().value_counts().head(1)
print(ans)

justin    14
Name: Player, dtype: int64


In [353]:
# PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라

data.groupby('Pos')[['PTS']].mean().sort_values('PTS')

PTS
Pos             
SF      7.270253
SG-SF   7.360000
C       7.690769
PF      7.737500
SG      7.810553
PG      8.325161
PF-SF   8.500000
SF-SG   8.660000
SG-PG   9.525000
C-PF   12.850000
PG-SG  22.000000

In [364]:
# G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라

data.sort_values(['Tm','G']).groupby('Tm').tail(1)['G'].mean()

77.51612903225806

In [366]:
#Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?

data[(data['Tm']=='MIA')&(data['Pos'].isin(['C','PF']))]['MP'].mean()

16.787499999999998

In [368]:
# 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?

data[data['G']>=data['G'].mean()*1.5]['3P'].mean()

1.3853658536585358

In [369]:
# Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?

a = data[data['Age']>=data['Age'].mean()]['G'].mean()
b = data[data['Age']<data['Age'].mean()]['G'].mean()

print(a-b)

3.787674551781862


In [374]:
# 평균나이가 가장 젊은 팀은 어디인가
data.groupby('Tm')['Age'].mean().sort_values(ascending=True).index[0]

'MEM'

In [375]:
# Pos그룹별 평균 MP값을 구하여라

data.groupby('Pos')['MP'].mean()

Pos
C        16.990000
C-PF     25.350000
PF       17.937500
PF-SF    27.300000
PG       19.547742
PG-SG    37.200000
SF       17.514557
SF-SG    20.340000
SG       18.554271
SG-PG    22.950000
SG-SF    19.620000
Name: MP, dtype: float64